In [1]:
import numpy as np
import tensorflow as tf
# sess.close()
sess = tf.InteractiveSession()

In [52]:
import os

In [2]:
GRID_DIM = 7
BATCH_SIZE = 16
NUM_BOXES = 2
NO_OBJECT_SCALE = .5
OBJECT_SCALE = 1.0
COORD_SCALE = 5.0
CLASS_SCALE = 1.0
CLASS_WEIGHTS = np.array([1.5,0.7,0.5], dtype='float32')
DEBUG = True

In [219]:
y_true = np.zeros((16,7,7,2,8))
y_true[...]=[.5,.5,.7,.7,1.,1.,0,0]
y_true = tf.convert_to_tensor(y_true, dtype=np.float32)

y_pred = np.zeros((16,7,7,2,8))
# y_pred[...,3,3,0,:]=[0.5,0.5,.5,.9,1.,.1,.0,0]
# y_pred[4:8,3,:,0,:]=[0.5,0.95,np.sqrt(0.4),np.sqrt(0.55),0.75,0.6,0.4,0]
# y_pred[:4,:,:,:,:]=[0.1,0.1,np.sqrt(4),np.sqrt(4),0.8,0.7,0.2,0.1]
# y_pred[...]=[0.6,0.8,0.4,0.1,0.9,0.9,0.4,0.1]
y_pred = tf.convert_to_tensor(y_pred, dtype=np.float32)

In [221]:
pred_box_xy = y_pred[..., :2] #+ cell_grid

### account for network predicting squares of w and h
sqrt_pred_box_wh = y_pred[..., 2:4]
pred_box_wh = tf.square(sqrt_pred_box_wh)    

### confidence should be in [0,1]
pred_box_conf = y_pred[..., 4]

### class probabilities
pred_box_class = y_pred[..., 5:]

"""
Load ground truth
"""
### x and y center of boxes
true_box_xy = y_true[..., 0:2] # relative position to the containing cell

### adjust w and h
sqrt_true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically
true_box_wh = tf.square(sqrt_true_box_wh)

### Find iou for conf given obj, else 0
true_wh_half = true_box_wh / 2.
true_mins    = true_box_xy - true_wh_half
true_maxes   = true_box_xy + true_wh_half

pred_wh_half = pred_box_wh / 2.
pred_mins    = pred_box_xy - pred_wh_half
pred_maxes   = pred_box_xy + pred_wh_half

intersect_mins  = tf.maximum(pred_mins,  true_mins)
intersect_maxes = tf.minimum(pred_maxes, true_maxes)
intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

union_areas = pred_areas + true_areas - intersect_areas
iou_scores  = tf.where(tf.less(tf.abs(union_areas), 1e-4), union_areas, tf.truediv(intersect_areas, union_areas)) 
true_box_conf = iou_scores * y_true[..., 4]

### adjust class probabilities
true_box_class = y_true[..., 5:]

"""
Determine the masks
"""
### coordinate mask: simply the position of the ground truth boxes (the predictors)
obj_mask_for_mult_attr = tf.expand_dims(y_true[..., 4], axis=-1)
obj_mask_for_one_attr = y_true[..., 4]
coord_mask =  obj_mask_for_mult_attr * COORD_SCALE
conf_mask_obj = obj_mask_for_one_attr * OBJECT_SCALE
conf_mask_no_obj = (1-obj_mask_for_one_attr) * NO_OBJECT_SCALE

"""
Summarize the loss
"""
# nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
# nb_conf_box_neg = tf.reduce_sum(tf.to_float(conf_mask_no_obj > 0.0))
# nb_conf_box_pos = tf.reduce_sum(tf.to_float(conf_mask_obj > 0.0))
# nb_obj = tf.reduce_sum(tf.to_float(obj_mask_for_one_attr > 0.0))

loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy) * coord_mask, axis=[1,2,3,4]) #/ (nb_coord_box + 1e-6) / 2.
loss_wh    = tf.reduce_sum(tf.square(sqrt_true_box_wh-sqrt_pred_box_wh) * coord_mask, axis=[1,2,3,4]) #/ (nb_coord_box + 1e-6) / 2.
loss_conf_neg = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask_no_obj, axis=[1,2,3]) #/ (nb_conf_box_neg + 1e-6) / 2.
loss_conf_pos = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask_obj, axis=[1,2,3]) #/ (nb_conf_box_pos + 1e-6) / 2.
loss_class = tf.reduce_sum(tf.square(true_box_class - pred_box_class)* obj_mask_for_mult_attr, axis=[1,2,3,4])#/(nb_obj + 1e-6)

loss = loss_xy + loss_wh + loss_conf_pos + loss_conf_neg + loss_class
print(tf.reduce_sum(loss).eval())

13171.215


In [220]:
tf.reduce_sum(tf.square(y_pred - y_true)).eval()

5456.622